# Setup

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from torchmetrics import Accuracy
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR

from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score, f1_score
from sklearn import metrics

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import random
from glob import glob
from tqdm import tqdm
from scipy.io import loadmat

import torch
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from datetime import datetime

In [3]:
from IPython.display import clear_output
import os, sys, shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm, trange
from glob import glob
import random
import cv2 as cv
import torch.nn.functional as F
import json
from sklearn.preprocessing import LabelEncoder

In [4]:
# main_data_dir =  "/media/mountHDD2"
print(os.getcwd())
save_dir = os.getcwd()
os.chdir("..")
# os.chdir("..")
main_dir = os.getcwd() 
print(os.listdir(main_dir))
print(save_dir)

/media/mountHDD2/linh/git/HeartResearch/Chapman Dataset/Reimplement
['Reimplement', 'Data_Explore', 'Diagnostics.xlsx', '.ipynb_checkpoints', '2D_model', '1D_Model', '2D_Loss']
/media/mountHDD2/linh/git/HeartResearch/Chapman Dataset/Reimplement


In [5]:
data_dir = "/media/mountHDD3/data_storage/biomedical_data/ecg_data/ECGDataDenoised"
label_file = main_dir + "/Diagnostics.xlsx"

In [6]:
ls = []
for file in glob(data_dir + "/*"):
    ls.append(file.split("/")[-1].split(".")[0].split("_")[0])
ls_list = pd.Series(ls)
ls_list.value_counts()

MUSE    10588
dtype: int64

In [7]:
diag_df = pd.read_excel(label_file)
label_df = diag_df[['FileName', 'Rhythm']]
label_df

,FileName,Rhythm
0,MUSE_20180120_121711_19000,SA
1,MUSE_20180120_121704_86000,SA
2,MUSE_20180113_125357_13000,SA
3,MUSE_20180113_134825_04000,SB
4,MUSE_20180115_123455_79000,SB
...,...,...
10641,MUSE_20181222_204246_47000,SVT
10642,MUSE_20180115_120332_79000,SA
10643,MUSE_20180712_152507_30000,AF
10644,MUSE_20180118_181350_17000,SA


In [8]:
# attribute_df = diag_df.iloc[:, 5:]
attribute_df = diag_df[['FileName', 'VentricularRate', 'AtrialRate', 'QRSDuration', 'QTInterval',
                       'QTCorrected', 'RAxis', 'TAxis', 'QRSCount', 'QOnset', 'QOffset', 'TOffset']]
attribute_df

,FileName,VentricularRate,AtrialRate,QRSDuration,QTInterval,QTCorrected,RAxis,TAxis,QRSCount,QOnset,QOffset,TOffset
0,MUSE_20180120_121711_19000,77,77,80,366,414,59,28,13,211,251,394
1,MUSE_20180120_121704_86000,68,68,80,402,427,29,53,11,220,260,421
2,MUSE_20180113_125357_13000,94,94,86,334,417,86,65,15,213,256,380
3,MUSE_20180113_134825_04000,58,58,102,432,424,2,60,10,211,262,427
4,MUSE_20180115_123455_79000,59,59,84,424,419,55,44,9,221,263,433
...,...,...,...,...,...,...,...,...,...,...,...,...
10641,MUSE_20181222_204246_47000,184,0,234,354,619,178,144,30,159,276,336
10642,MUSE_20180115_120332_79000,69,69,76,400,428,77,63,12,219,257,419
10643,MUSE_20180712_152507_30000,102,340,90,398,518,48,210,17,211,256,410
10644,MUSE_20180118_181350_17000,77,77,102,398,450,-16,30,13,210,261,409


In [9]:
unique_values = np.unique(label_df["Rhythm"].values.tolist())
print(unique_values)

['AF' 'AFIB' 'AT' 'AVNRT' 'AVRT' 'SA' 'SAAWR' 'SB' 'SR' 'ST' 'SVT']


In [10]:
# data_paths = []
# for file in glob(data_dir +"/*"):
#     data_paths.append(file)

In [11]:
# data_dict = {
#     idx : [] for idx in unique_values
# }

# for data_path in data_paths:
#     filename = data_path.split("/")[-1].split(".")[0]
    
#     # Get the corresponding 'Rhythm' values for the filename
#     rhythm_values = label_df[label_df["FileName"] == filename]["Rhythm"].values
    
#     if rhythm_values.size == 1:
#         _cls = rhythm_values.item()  # Safe to use .item() if there's exactly one match
#     elif rhythm_values.size > 1:
#         _cls = rhythm_values[0]  # Take the first element if there are multiple matches
#     else:
#         continue  # Skip if no matches are found
    
#     data_dict[_cls].append(data_path)

# for key in data_dict:
#     print(f"{key}->{len(data_dict[key])}")

In [12]:
label_encoder = LabelEncoder()
label_df['Rhythm'] = label_encoder.fit_transform(label_df['Rhythm'])
print(label_df)

                         FileName  Rhythm
0      MUSE_20180120_121711_19000       5
1      MUSE_20180120_121704_86000       5
2      MUSE_20180113_125357_13000       5
3      MUSE_20180113_134825_04000       7
4      MUSE_20180115_123455_79000       7
...                           ...     ...
10641  MUSE_20181222_204246_47000      10
10642  MUSE_20180115_120332_79000       5
10643  MUSE_20180712_152507_30000       0
10644  MUSE_20180118_181350_17000       5
10645  MUSE_20180116_121646_28000       9

[10646 rows x 2 columns]


/tmp/ipykernel_353957/787631178.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_df['Rhythm'] = label_encoder.fit_transform(label_df['Rhythm'])


In [13]:
filenames = label_df["FileName"].values.tolist()

In [14]:
unique_values = np.unique(label_df["Rhythm"].values.tolist())
print(unique_values)

[ 0  1  2  3  4  5  6  7  8  9 10]


In [15]:
data_paths = []
for file in glob(data_dir +"/*"):
    data_paths.append(file)
print(len(data_paths))

10588


In [16]:
ratio = [0.8, 0.1]

train_index = int(len(data_paths)*ratio[0])
valid_index = int(len(data_paths)*(ratio[0]+ratio[1]))
print(train_index)
train_mat_paths = data_paths[:train_index]
valid_mat_paths = data_paths[train_index:valid_index]
print(len(train_mat_paths))

train_label = label_df.iloc[:train_index,:]
valid_label = label_df.iloc[train_index:valid_index,:]

8470
8470


In [17]:
print(len(train_label))
print(len(valid_label))

8470
1059


In [18]:
label_df[label_df["FileName"] == filenames]["Rhythm"].values

array([5, 5, 5, ..., 0, 5, 9])

In [19]:
ECG_feature = attribute_df[attribute_df["FileName"] == 'MUSE_20180120_121711_19000'].iloc[:, 1:].to_numpy()[0]
print(ECG_feature)

[ 77  77  80 366 414  59  28  13 211 251 394]


 # Data Loader

In [20]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.nn.functional import one_hot

In [21]:
# data = []
class HeartData(Dataset):
    def __init__(self, data_paths, label_df, attribute_df):
        self.data_paths = data_paths
        random.shuffle(self.data_paths)
        self.label_df = label_df
        self.attribute_df = attribute_df
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
        self.transform = transforms.Compose([
            transforms.Resize((224, 224), antialias=None),
            normalize
        ])

    def __getitem__(self, idx):
        data_path = self.data_paths[idx]        
        # data = loadmat(data_path)['ECG'][0][0][2]
        data = pd.read_csv(data_path, header = None)
        data = data.values.T
        clip_data = data[:, 500:3000]
        clip_data = torch.tensor(clip_data, dtype=torch.float32)
        # normalized_data = (clip_data - clip_data.min()) / (clip_data.max() - clip_data.min())
        # grayscale_images = (normalized_data * 255)
        # grayscale_images = grayscale_images.unsqueeze(0).unsqueeze(0) # (1, 1, h, w)
        # resized_images = F.interpolate(grayscale_images, size=(9*4,2500), mode='bilinear', align_corners=True)
        # resized_images = resized_images.squeeze(0).squeeze(0)
        # torch_data = resized_images.unsqueeze(0).repeat(3, 1, 1)
        # torch_data_resize = self.transform(torch_data)

        filename = data_path.split("/")[-1].split(".")[0]
        label = self.label_df[self.label_df["FileName"] == filename]["Rhythm"].values.item()
        ECG_feature = self.attribute_df[self.attribute_df["FileName"] == filename].iloc[:, 1:].to_numpy()[0]

        return clip_data, label, ECG_feature

    def __len__(self):
        return len(self.data_paths)    

In [22]:
train_ds = HeartData(train_mat_paths, label_df, attribute_df)
valid_ds = HeartData(valid_mat_paths, label_df, attribute_df)
print(len(valid_ds))

1059


In [23]:
device = torch.device("cuda")
batch_size = 64

traindl = DataLoader(
    train_ds,
    batch_size=batch_size, 
    shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

validdl = DataLoader(
    valid_ds,
    batch_size=29, 
    # shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

# Model 

In [24]:
import torch
import torch.nn as nn

class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        # Input size: (batch_size, 12, 300)

        self.conv1 = nn.Conv1d(12, 64, kernel_size=21, stride=11)
        # self.pca1 = nn.Linear(64,11)
        self.layer1 = nn.Sequential(
            nn.MaxPool1d(2),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            nn.Dropout(0.3)
        )
         

        self.conv6 = nn.Conv1d(64, 128, kernel_size=7, stride=1)
        # self.pca6 = nn.Linear(128,11)
        self.layer2 = nn.Sequential(
            nn.MaxPool1d(2),
            nn.BatchNorm1d(128),
            nn.Conv1d(128, 256, kernel_size=5, stride=1),
            nn.MaxPool1d(2)
        )
        
        self.conv11 = nn.Conv1d(256, 512, kernel_size=13, stride=1)
        # self.pca11 = nn.Linear(512,11)
        self.layer3 = nn.Sequential(
            nn.ConvTranspose1d(512, 256, kernel_size=7, stride=1),
            nn.Dropout(0.3)
        )

        self.conv14 = nn.Conv1d(256, 256, kernel_size=9, stride=1)
        self.pca14 = nn.Linear(256,11)
        self.pool4 = nn.MaxPool1d(2)

        self.lstm = nn.LSTM(input_size=256, hidden_size=128, batch_first=True)
        self.fc = nn.Linear(128, 11)

    def forward(self, x):
        # Input: (batch_size, 12, 300)
        x = self.conv1(x)  # (batch_size, 64, ?)
        x = self.layer1(x)  # (batch_size, 64, ?)
    
        x = self.conv6(x)  # (batch_size, 128, ?)
        x = self.layer2(x)  # (batch_size, 256, ?)
    
        x = self.conv11(x)  # (batch_size, 512, ?)
        x = self.layer3(x)  # (batch_size, 256, ?)
    
        x_feature = self.conv14(x)  # (batch_size, 256, ?)
        x = self.pool4(x_feature)
    
        # Flatten for Linear layer
        x = x.permute(0, 2, 1)  # (batch_size, seq_len, 256)
        # pca14 = x.reshape(-1, 256)  # (batch_size * seq_len, 256)
        # pca14 = self.pca14(x)  # Linear layer: (batch_size * seq_len, 11)
        
        # # Reshape back to batch format (if needed for downstream tasks)
        # seq_len = pca14.size(0) // x.size(0)  # Compute seq_len
        # pca14 = pca14.view(-1, seq_len, 11)  # (batch_size, seq_len, 11)
    
        # Use pooled feature maps
        # x = self.pool4(x)  # If further processing is required
        # x = x.permute(0, 2, 1)  # (batch_size, seq_len, 256)
    
        x, _ = self.lstm(x)  # LSTM expects (batch_size, seq_len, input_size)
        x = x[:, -1, :]  # Take the last time step's output
        out = self.fc(x)  # Final output layer
        return out, x_feature

In [25]:
a = torch.rand(1,12,2500)
out_DNN, feature = DNN()(a)
print(feature.shape)

torch.Size([1, 256, 10])


# Training

In [26]:
epoch = 2
lr = 0.001 # lr = 0.001 Acc: 0.821875 lr = 0.0005 Acc: 0.825 Focal_loss Acc: 0.828
best_acc = 0
best_ep = 0

model = DNN()
model.to(device)
optimizer = Adam(model.parameters(), lr=lr)
scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=epoch*len(traindl))

In [27]:
class FocalClassifierV0(nn.Module):
    def __init__(self, gamma=0.3):
        super().__init__()
        
        self.gamma = gamma
        self.act = nn.LogSoftmax(dim=1)
    
    def forward(self, pred, target):

        logits = self.act(pred)

        B, C = tuple(logits.size())

        entropy = torch.pow(1 - logits, self.gamma) * logits * F.one_hot(target, num_classes=C).float()

        return (-1 / B) * torch.sum(entropy)

focalloss_fn = FocalClassifierV0()
loss_fn = nn.CrossEntropyLoss()
# checkpoint_folder = "run_efficientB0_heatmap_gamma0.5_lr0.0005"
checkpoint_folder = "run_deep_feature_0.3_0.01_lr00005"
# checkpoint_folder = "run_proposed_gamma5_0.01_lr0001_10"

In [28]:
def loss_plot(train_losses, val_losses, n_epochs, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_loss_dir = run_dir + "/save_losses"
    if not os.path.exists(save_loss_dir):
        os.mkdir(save_loss_dir)
    save_fig_losses = os.path.join(save_loss_dir, f"plot_losses_epoch{n_epochs}_{now}.png")  

    plt.figure()
    plt.plot(range(epoch), np.array(train_losses), label='Train Loss')
    plt.plot(range(epoch), np.array(val_losses), label='Test Loss')
    plt.xlabel('Epoch')
    # plt.xticks()
    plt.ylabel('Loss')
    plt.title('Training and Test Loss')
    plt.legend()
    plt.savefig(save_fig_losses)
    

def acc_plot(train_cls_acc, val_cls_acc, n_epochs, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
#         save_dir = run_dir + f"/{now}"
#         if not os.path.exists(save_dir):
#             os.mkdir(save_dir)
    save_acc_dir = run_dir + "/save_acc"
    if not os.path.exists(save_acc_dir):
        os.mkdir(save_acc_dir)
    save_fig_acc = os.path.join(save_acc_dir, 'plot_acc_epoch{}_{}.png'.format(n_epochs, now))  

    plt.figure()
    plt.plot(range(epoch), np.array(train_cls_acc), label='Train Accuracy')
    plt.plot(range(epoch), np.array(val_cls_acc), label='Test Accuracy')
    plt.xlabel('Epoch')
    # plt.xticks
    plt.ylabel('Accuracy')
    plt.title('Training and Test Accuracy')
    plt.legend()
    plt.savefig(save_fig_acc)

def checkpoint(valid_class_acc, 
               val_total_loss,
               old_valid_class_acc,
               old_valid_loss,
               epoch, 
               model,
               optimizer,
               check_folder
#                    logs
              ):

    if valid_class_acc >= old_valid_class_acc and val_total_loss <= old_valid_loss:
        old_valid_class_acc = valid_class_acc
        old_valid_loss = val_total_loss
        save_dict = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_total_loss,
            'test_acc': valid_class_acc
        }

     # Saving best model
        now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
        run_dir = save_dir + f"/{check_folder}"
        if not os.path.exists(run_dir):
            os.mkdir(run_dir)
    #         save_dir = run_dir + f"/{now}"
    #         if not os.path.exists(save_dir):
    #             os.mkdir(save_dir)
        save_best_model_dir = run_dir + "/save_best_model"
        if not os.path.exists(save_best_model_dir):
            os.mkdir(save_best_model_dir)
        save_best_model_path = save_best_model_dir + f"/{save_dict['loss']:>7f}_{save_dict['test_acc']:>7f}_{now}.pt"
        torch.save(save_dict, save_best_model_path)
        
def classification_report_csv(report, auc, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_report_dir = run_dir + "/save_classification_report"
    if not os.path.exists(save_report_dir):
        os.mkdir(save_report_dir)
        
    report_data = report['macro avg']
    del report_data['support']
    report_data.update({'auc': auc})
#     print(type(report_data))
#     print(report_data)

#     dataframe = pd.DataFrame.from_dict(report_data, orient='index')
#     save_report_file = save_report_dir + f"/classification_report_{now}.csv"
#     dataframe.to_csv(save_report_file, index = False)
    with open(save_report_dir + f"/cls_report_{now}.json", "w") as outfile: 
        json.dump(report_data, outfile)
    
def acc_loss_json(log_dict, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_json_dir = run_dir + "/save_acc_loss_json"
    if not os.path.exists(save_json_dir):
        os.mkdir(save_json_dir) 
    save_acc_loss_file = save_json_dir + f"/acc_loss_{now}.json"
    with open(save_acc_loss_file, "w") as outfile: 
        json.dump(log_dict, outfile)

In [29]:
log_dict = {
    "train": {
        "acc": [],
        "loss": []
    },
    "valid": {
        "acc": [],
        "loss": []
    }
}

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
class_la = []
for i in range (11):
    class_la.append(i)
for i in range (len(class_la)):
    class_la[i] = str(class_la[i])
train_losses = []
train_acc_plot = []
val_losses = []
val_acc_plot = []
old_valid_class_acc = 0
old_valid_loss = 1e23
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
pca = PCA(n_components=11)

In [31]:
train_feature_list = []
train_label_list = []
for e in range(epoch):
    model.train()
    print(f"Epoch: {e+1}")
    batch_cnt = 0
    total_loss = 0
    correct = 0
    for batch, (train_sig, train_label, feature_train) in tqdm(enumerate(traindl)):
        
        batch_cnt = batch
        train_sig = train_sig.to(device)
        # print(train_sig.shape)
        train_label = train_label.to(device)
        # print(train_label.shape)

        pred, feature_map_train = model(train_sig)
        X_signal_train_features = feature_map_train.cpu().detach().numpy()
        if e == 1:
            X_signal_train_pca = pca.fit_transform(X_signal_train_features.reshape(X_signal_train_features.shape[0], -1))
            # print(X_signal_train_pca.shape)
            X_train_combined = torch.cat((torch.tensor(X_signal_train_pca), feature_train), dim=1).cpu().detach().numpy()
            # print(X_train_combined.shape)
            train_feature_list.extend(X_train_combined)
            train_label_list.extend(train_label.cpu())
        loss = loss_fn(pred, train_label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

    test_feature_list = []
    test_label_list = []
    batch_cnt = 0
    val_total_loss = 0
    val_correct = 0
    accuracy = 0
    model.eval()
    y_true_list = [] 
    pred_list = []
    with torch.no_grad():
        for batch, (valid_sig, valid_label, feature_test) in tqdm(enumerate(validdl)):
            batch_cnt = batch
            valid_label = valid_label.to(device)
            valid_sig = valid_sig.to(device)  
            pred_test, feature_map_test = model(valid_sig)
            X_signal_test_features = feature_map_test.cpu().detach().numpy()
            # y_true_list.extent(valid_label)
            # X_signal_train_pca = pca.fit_transform(X_signal_train_features.reshape(X_signal_train_features.shape[0], -1))
            if e == 1:
                print(X_signal_test_features.shape)
                print(X_signal_test_features.reshape(X_signal_test_features.shape[0], -1).shape)
                X_signal_test_pca = pca.fit_transform(X_signal_test_features.reshape(X_signal_test_features.shape[0], -1))
                
            # X_train_combined = np.hstack([X_signal_train_pca, feature_train])
            # X_test_combined = np.hstack([X_signal_test_pca, feature_test])
            # X_train_combined = torch.cat(( torch.tensor(X_signal_train_pca), feature_train), dim=0)
                X_test_combined = torch.cat(( torch.tensor(X_signal_test_pca), feature_test), dim=1).cpu().detach().numpy()
                test_feature_list.extend(X_test_combined)
                test_label_list.extend(valid_label.cpu())  

Epoch: 1


133it [00:03, 37.16it/s]
37it [00:00, 62.28it/s]

Epoch: 2



133it [00:07, 18.48it/s]
1it [00:00,  3.06it/s]

(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)


...

18it [00:00, 33.50it/s]

(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)


32it [00:00, 50.58it/s]

(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(29, 256, 10)
(29, 2560)
(15, 256, 10)
(15, 2560)


37it [00:01, 29.58it/s]


In [32]:
print(len(train_feature_list))
print(len(train_label_list))

8470
8470


In [36]:
rf_model.fit(train_feature_list, train_label_list)
        
        # Evaluate the model
y_pred = rf_model.predict(test_feature_list)
# print(y_pred)
# pred_list.append(y_pred)
accuracy = accuracy_score(test_label_list, y_pred)
f1 = f1_score(test_label_list, y_pred, average = "macro")
print(f"Random Forest Accuracy: {accuracy * 100:.2f}%")
print(f"f1: {f1 * 100:.2f}%")

Random Forest Accuracy: 85.08%
f1: 45.24%


In [ ]:
# print(reports.keys())
# print(reports['macro avg'])

# Evaluation